In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
cci = pd.read_excel("/Users/yena/Desktop/SCH/척척석사/EMR related/kcd7_cci_eci.xlsx") # cci 파일 불러오기
cci = cci.rename(columns = {'표준코드': 'ICD10'})   # 컬럼 이름 '표준코드' --> 'ICD10'
cci = cci[['ICD10', 'cci_wscore']]  # 표준코드, score 만 분리
cci = cci.rename({'cci_wscore':'CCI_WSCORE'}, axis=1)    # 컬럼 이름 'cci_wscore' --> 'CCI_WSCORE'
cci = cci.rename({'ICD10':'icd_10'}, axis=1)     # 컬럼 이름 'ICD10' --> 'icd_10'
cci = cci.drop_duplicates() # 중복 제거

### 몸무게, 키

In [3]:
hw = pd.read_excel('/Users/yena/Desktop/SCH/척척석사/EMR related/낙상데이터/천안/(수정)1642636158_20170701부터 20211130 간호통합병동재원 환자기준 환자기초정보 및 상병정보등 자료추출_20220119.xlsx')
hw.head(2)

,환자번호,입원일자,퇴원일자,DSCHDD_YN,성별,나이,키,몸무게,"주상병 여부\n(M : 주상병, S:부상병)",상병코드,상병명,ICD10 상병코드,ICD10 상병코드명,병동코드,진료과
0,1579661,20200531,20200723.0,Y,M,17,NaN,37.0,M,J189.000.01,폐렴,J189,폐렴,BA02B,MP
1,1579661,20200531,20200723.0,Y,M,17,NaN,37.0,M,J189.000.01,폐렴,J189,폐렴,BA03,MP


In [4]:
hw_df = hw[['환자번호', '성별', '나이', '키', '몸무게']].drop_duplicates().fillna(0)
hw_df.head()

,환자번호,성별,나이,키,몸무게
0,1579661,M,17,0.0,37.0
20,767152,M,85,0.0,49.5
36,681799,F,56,0.0,55.0
39,6822,F,65,0.0,64.0
45,323972,M,59,0.0,70.0


In [5]:
len(hw_df['환자번호'].unique())

23934

In [6]:
hw_df = pd.DataFrame(hw_df.groupby(['환자번호', '성별'])['나이', '키', '몸무게'].mean()).reset_index().round(2)

In [7]:
hw_df.to_csv('천안_기초정보.csv')

In [8]:
hw_df

,환자번호,성별,나이,키,몸무게
0,16,M,65.67,167.33,72.07
1,22,F,77.00,153.10,71.00
2,28,F,62.00,158.50,45.00
3,76,F,61.00,161.60,61.70
4,115,M,62.00,0.00,64.00
...,...,...,...,...,...
23945,3113592,M,53.20,167.22,49.60
23946,3113602,M,21.00,172.00,70.00
23947,3113619,M,33.00,175.00,99.00
23948,3113889,M,63.00,0.00,69.80


### 질병

In [9]:
hw_cci = hw[['환자번호','주상병 여부\n(M : 주상병, S:부상병)','ICD10 상병코드','입원일자']].drop_duplicates()
hw_cci

,환자번호,"주상병 여부\n(M : 주상병, S:부상병)",ICD10 상병코드,입원일자
0,1579661,M,J189,20200531
2,1579661,S,E870,20200531
4,1579661,S,E039,20200531
6,1579661,M,J690,20200531
8,1579661,S,G4080,20200531
...,...,...,...,...
266017,1583554,S,I200,20211107
266018,1601418,M,C3499,20211112
266020,1601418,S,Z115,20211112
266026,1601418,S,J4499,20211112


In [10]:
hw_cci_v2 = hw_cci.copy()

In [11]:
hw_cci = hw_cci[hw_cci['주상병 여부\n(M : 주상병, S:부상병)']=='M'].drop_duplicates()

In [12]:
hw_cci_v2 = hw_cci_v2[hw_cci_v2['주상병 여부\n(M : 주상병, S:부상병)']=='S'].drop_duplicates()

In [13]:
# hw_cci = hw_cci.drop(columns='주상병 여부\n(M : 주상병, S:부상병)')
hw_cci = hw_cci.rename(columns={'ICD10 상병코드':'주상병'})
# hw_cci_v2 = hw_cci_v2.drop(columns='주상병 여부\n(M : 주상병, S:부상병)')
hw_cci_v2 = hw_cci_v2.rename(columns={'ICD10 상병코드':'부상병'})

In [14]:
notice = ['F000', 'F001', 'F002', 'F009', 'F010', 'F011', 'F012', 'F013', 'F018',\
     'F019','F020','F021','F022','F023','F024','F028','F03','F051','K7043','F104'\
        'F445', 'F525', 'R252', 'R560', 'R568', 'T672']

In [15]:
import numpy as np

spe = []

for i  in hw_cci['주상병'].tolist():
    if i in notice:
        spe.append('있음')
    else:
        spe.append(np.nan)

hw_cci['주진단_특수'] = spe

In [16]:
spe = []

for i  in hw_cci_v2['부상병'].tolist():
    if i in notice:
        spe.append('있음')
    else:
        spe.append(np.nan)

hw_cci_v2['부진단_특수'] = spe

In [17]:
hw_cci_v2['부진단_특수'].value_counts()

있음    1477
Name: 부진단_특수, dtype: int64

In [18]:
hw_cci_df = hw_cci_v2.merge(hw_cci,on=['환자번호','입원일자']).drop_duplicates()

In [19]:
hw_cci_df

,환자번호,"주상병 여부\n(M : 주상병, S:부상병)_x",부상병,입원일자,부진단_특수,"주상병 여부\n(M : 주상병, S:부상병)_y",주상병,주진단_특수
0,1579661,S,E870,20200531,NaN,M,J189,NaN
1,1579661,S,E870,20200531,NaN,M,J690,NaN
2,1579661,S,E039,20200531,NaN,M,J189,NaN
3,1579661,S,E039,20200531,NaN,M,J690,NaN
4,1579661,S,G4080,20200531,NaN,M,J189,NaN
...,...,...,...,...,...,...,...,...
189667,1583554,S,I201,20211107,NaN,M,I214,NaN
189668,1583554,S,I200,20211107,NaN,M,I214,NaN
189669,1601418,S,Z115,20211112,NaN,M,C3499,NaN
189670,1601418,S,J4499,20211112,NaN,M,C3499,NaN


In [20]:
hw_cci_df = hw_cci_df[['환자번호', '부상병', '입원일자', '부진단_특수', '주상병', '주진단_특수']]
hw_cci_df

,환자번호,부상병,입원일자,부진단_특수,주상병,주진단_특수
0,1579661,E870,20200531,NaN,J189,NaN
1,1579661,E870,20200531,NaN,J690,NaN
2,1579661,E039,20200531,NaN,J189,NaN
3,1579661,E039,20200531,NaN,J690,NaN
4,1579661,G4080,20200531,NaN,J189,NaN
...,...,...,...,...,...,...
189667,1583554,I201,20211107,NaN,I214,NaN
189668,1583554,I200,20211107,NaN,I214,NaN
189669,1601418,Z115,20211112,NaN,C3499,NaN
189670,1601418,J4499,20211112,NaN,C3499,NaN


In [21]:
spe_df = hw_cci_df[hw_cci_df['주진단_특수']=='있음'].drop(columns='입원일자').drop_duplicates().append(hw_cci_df[hw_cci_df['부진단_특수']=='있음'].drop(columns='입원일자').drop_duplicates())
spe_df['disease'] ='있음'
spe_df = spe_df[['환자번호', 'disease']].drop_duplicates()
spe_df

,환자번호,disease
4946,10110,있음
8462,1512572,있음
46209,1205319,있음
53899,1608531,있음
89733,219863,있음
...,...,...
189475,1666081,있음
189486,1657556,있음
189528,877058,있음
189541,11661,있음


In [22]:
hw_cci_df = hw_cci_df.rename(columns = {'주상병':'icd_10'})
hw_cci_df 

,환자번호,부상병,입원일자,부진단_특수,icd_10,주진단_특수
0,1579661,E870,20200531,NaN,J189,NaN
1,1579661,E870,20200531,NaN,J690,NaN
2,1579661,E039,20200531,NaN,J189,NaN
3,1579661,E039,20200531,NaN,J690,NaN
4,1579661,G4080,20200531,NaN,J189,NaN
...,...,...,...,...,...,...
189667,1583554,I201,20211107,NaN,I214,NaN
189668,1583554,I200,20211107,NaN,I214,NaN
189669,1601418,Z115,20211112,NaN,C3499,NaN
189670,1601418,J4499,20211112,NaN,C3499,NaN


In [23]:
result = hw_cci_df.merge(cci, on=['icd_10'],how='inner').drop_duplicates()

In [24]:
result = result.rename(columns={'icd_10':'주진단'})
result = result.rename(columns={'부상병':'icd_10'})

In [25]:
result2 =result.merge(cci, on=['icd_10'],how='inner').drop_duplicates()
result2['total_cci'] = result2['CCI_WSCORE_x'] + result2['CCI_WSCORE_y']

result2 = result2.drop(columns=['주진단','icd_10','CCI_WSCORE_x','CCI_WSCORE_y',\
    '주진단_특수', '부진단_특수']).drop_duplicates()
result2

,환자번호,입원일자,total_cci
0,1579661,20200531,0.0
1,1604637,20190916,0.0
2,1446073,20190218,0.0
3,850473,20170630,0.0
4,1510847,20180213,0.0
...,...,...,...
185711,1717102,20211126,0.0
185714,1677039,20210303,0.0
185715,1502962,20171215,0.0
185725,1474052,20170709,0.0


In [26]:
patient_cci = pd.DataFrame(result2.groupby(['환자번호'])['total_cci'].mean()).reset_index().drop_duplicates().round(1)
patient_cci.merge(spe_df, on=['환자번호'], how='outer')

,환자번호,total_cci,disease
0,16,2.9,NaN
1,22,0.5,NaN
2,28,0.0,NaN
3,76,2.0,NaN
4,115,0.0,NaN
...,...,...,...
19611,3113619,0.0,NaN
19612,3113889,0.5,NaN
19613,578322,NaN,있음
19614,195709,NaN,있음


In [27]:
len(patient_cci['환자번호'].unique())

19613

In [28]:
patient_cci.to_csv('천안_cci.csv')